In [1]:
import os 
import json
import pprint 
import tweepy

In [5]:
#Twitter API credentials
api_key = "KLRpHp13jm62W7DK0QT4eX3d7"
api_secret = "30S9ZtJFYS6MLFUwdW7DVUgU6a0AXJ9ZqLw7Sey0Jr4ZU95XcU"
access_token = "1455882501130493955-uGFA6LQQvtxjQC9b7UoD7DCnjZhscy"
access_secret = "gEDVqti3J4gt0NzqM8PFdOYHrPQFkppwVS0redI8JzNbd"

In [6]:
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
if(api.verify_credentials):
    print ('Authentication completed successfully!')

Authentication completed successfully!


# JSON

In [8]:
data_folder = "data"

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}


# FOLLOWERS e FOLLOWING

In [13]:
def findUsers(userlist,mode): #mode: 0 = followers, 1 = following
    quantity=20
    foundlist={}
    l2list = {}

    for user in userlist:
        print(f"Processing user {user}")

        found = []
        l2list[user]=[]

        if mode==0:
            usergroup=tweepy.Cursor(api.followers,screen_name = user ).items(quantity)
        else:
            usergroup=tweepy.Cursor(api.friends,screen_name = user ).items(quantity)

        for item in usergroup:

            json_data = item._json

            found_user = {}
            found_user["id"] = json_data["id"]
            found_user["name"] = json_data["name"]
            found_user["screen_name"] = json_data["screen_name"]
            found_user["location"] = json_data["location"]
            found_user["followers_count"] = json_data["followers_count"]
            found_user["friends_count"] = json_data["friends_count"]
            found_user["protected"] = json_data["protected"]


            if item.friends_count < 1000 and item.followers_count<1000 and json_data["protected"] == False and (found_user not in found):
                found.append(found_user)
                l2list[user].append(item.screen_name)
                print(item.screen_name)

            if len(found) >= 5:
                break

        foundlist[user] = found

        if mode==0:
            serialize_json(data_folder, user+"_followers.json", foundlist)
        else:
            serialize_json(data_folder, user+"_following.json", foundlist)

    return l2list

l1_followers=findUsers(["mizzaro","Miccighel_"],0)
l1_following=findUsers(["mizzaro","Miccighel_"],1)

l2a_followers_mizzaro=findUsers(l1_followers["mizzaro"],0)
l2a_following_mizzaro=findUsers(l1_followers["mizzaro"],1)

l2a_followers_miccighel=findUsers(l1_followers["Miccighel_"],0)
l2a_following_miccighel=findUsers(l1_followers["Miccighel_"],1)

l2b_followers_mizzaro=findUsers(l1_following["mizzaro"],0)
l2b_following_mizzaro=findUsers(l1_following["mizzaro"],1)

l2b_followers_miccighel=findUsers(l1_following["Miccighel_"],0)
l2b_following_miccighel=findUsers(l1_following["Miccighel_"],1)

Processing user mizzaro
OlegZendel
LyndaSaidLhadj
acai2021
rodger_benham
ColoursRiot
Data serialized to path: data/mizzaro_followers.json
Processing user Miccighel_
Il_Bale_
Outsider289
mizzaro
EUROARTISTS
fra_bomba6
Data serialized to path: data/Miccighel__followers.json
Processing user OlegZendel
bevan_koopman
Marwah_k
ivrik
Dars2022_Sympo
ameertawfik
Data serialized to path: data/OlegZendel_followers.json
Processing user LyndaSaidLhadj
Christo12656926
gael_dias
Nthakur20
AthmaniAmine
iiia_unipd
Data serialized to path: data/LyndaSaidLhadj_followers.json
Processing user acai2021
TilburgU_DCA
srj_krishna
gabriellearuta
lesleywilton
sencerdeniz
Data serialized to path: data/acai2021_followers.json
Processing user rodger_benham
MaksudaBilkis
jparapar
ameertawfik
Jinxuan16807201
Sannisthsoni
Data serialized to path: data/rodger_benham_followers.json
Processing user ColoursRiot


Rate limit reached. Sleeping for: 516


KeyboardInterrupt: 

# Dettagli del profilo dei followers di primo e secondo livello

In [ ]:
def userDetails(userList):
    
    for user in userList:
        
        raw_data = api.get_user(user)
        json_data = raw_data._json
        profile_data = {}
        profile_data["id"] = json_data["id"]
        profile_data["name"] = json_data["name"]
        profile_data["screen_name"] = json_data["screen_name"]
        profile_data["location"] = json_data["location"]
        profile_data["description"] = json_data["description"]
        profile_data["followers_count"] = json_data["followers_count"]
        profile_data["friends_count"] = json_data["friends_count"]
        serialize_json(data_folder, "1_"+profile_data["screen_name"]+"_profile_data.json", profile_data)
        pp.pprint(profile_data)
        
userDetails(l1_followers)
userDetails(l2a_followers_mizzaro)
userDetails(l2a_followers_miccighel)
userDetails(l2b_followers_mizzaro)
userDetails(l2b_followers_miccighel)

# Ultimi 5 tweet di ciascun follower di primo e secondo livello

In [ ]:
def getTweets(userList):
    
    for user in userList:
        
        tweets = []
        for item in tweepy.Cursor(
                api.user_timeline, 
                screen_name=user,
                tweet_mode="extended"
            ).items(5):
            raw_tweet = item._json
            tweet = {}
            tweet["id"]= raw_tweet["id"]
            tweet["created_at"] = raw_tweet["created_at"]
            tweet["full_text"] = raw_tweet["full_text"],
            tweet["favorite_count"]= raw_tweet["favorite_count"]
            tweets.append(tweet)
        serialize_json(data_folder, user["screen_name"]_+"recent_tweets.json", tweets)
        pp.pprint(tweets)
        
getTweets(l1_followers)
getTweets(l2a_followers_mizzaro)
getTweets(l2a_followers_miccighel)
getTweets(l2b_followers_mizzaro)
getTweets(l2b_followers_miccighel)

# Rete sociale